### Инструкция по выполнению проекта

* 1.Загрузите и подготовьте данные. Поясните порядок действий.
* 2.Обучите и проверьте модель для каждого региона:
* 2.1. Разбейте данные на обучающую и валидационную выборки в
соотношении 75:25.
* 2.2. Обучите модель и сделайте предсказания на валидационной
выборке.
* 2.3. Сохраните предсказания и правильные ответы на валидационной
выборке.
* 2.4. Напечатайте на экране средний запас предсказанного сырья и
RMSE модели.
* 2.5. Проанализируйте результаты.
* 3.Подготовьтесь к расчёту прибыли:
* 3.1. Все ключевые значения для расчётов сохраните в отдельных
переменных.
* 3.2. Рассчитайте достаточный объём сырья для безубыточной
разработки новой скважины. Сравните полученный объём сырья со
средним запасом в каждом регионе.
* 3.3. Напишите выводы по этапу подготовки расчёта прибыли.
* 4.Напишите функцию для расчёта прибыли по выбранным скважинам и
предсказаниям модели:
* 4.1. Выберите скважины с максимальными значениями предсказаний.
* 4.2. Просуммируйте целевое значение объёма сырья, соответствующее
этим предсказаниям.
* 4.3. Рассчитайте прибыль для полученного объёма сырья.
* 5.Посчитайте риски и прибыль для каждого региона:
* 5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти
распределение прибыли.
* 5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск
убытков. Убыток — это отрицательная прибыль.
* 5.3. Напишите выводы: предложите регион для разработки скважин и
обоснуйте выбор.

### Описание данных

* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами
признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

### Условия задачи:

* Для обучения модели подходит только линейная регрессия (остальные
— недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью
машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода.
Доход с каждой единицы продукта составляет 450 тыс. рублей,
поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых
вероятность убытков меньше 2.5%. Среди них выбирают регион с
наибольшей средней прибылью.

*Данные синтетические: детали контрактов и характеристики месторождений
не разглашаются.*


In [3]:
# имортированные библиотеки:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

## Подготовка данных

In [6]:
try:
    data1 = pd.read_csv('D:\geo_data_0.csv')
    data2 = pd.read_csv('D:\geo_data_1.csv')
    data3 = pd.read_csv('D:\geo_data_2.csv')
except:
    data1 = pd.read_csv('datasets/geo_data_0.csv')
    data2 = pd.read_csv('datasets/geo_data_1.csv')
    data3 = pd.read_csv('datasets/geo_data_2.csv')
    
def data_print(one, two, three):
    print('Head')
    print(one.head(), two.head(), three.head(), sep='\n', end='\n'*2)
    print('Info')
    print(one.info(), two.info(), three.info(), sep='\n')
      
data_print(data1, data2, data3)

Head
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746

Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 en

In [7]:
print(
    data1['id'].duplicated().sum(),
    data2['id'].duplicated().sum(),
    data3['id'].duplicated().sum()
)

data1 = data1.drop_duplicates(['id'])
data2 = data2.drop_duplicates(['id'])
data3 = data3.drop_duplicates(['id'])

10 4 4


In [8]:
data1 = data1.drop(['id'], axis=1)
data2 = data2.drop(['id'], axis=1)
data3 = data3.drop(['id'], axis=1)

print(data_print(data1, data2, data3))

Head
         f0        f1        f2     product
0  0.705745 -0.497823  1.221170  105.280062
1  1.334711 -0.340164  4.365080   73.037750
2  1.022732  0.151990  1.419926   85.265647
3 -0.032172  0.139033  2.978566  168.620776
4  1.988431  0.155413  4.751769  154.036647
          f0         f1        f2     product
0 -15.001348  -8.276000 -0.005876    3.179103
1  14.272088  -3.475083  0.999183   26.953261
2   6.263187  -5.948386  5.001160  134.766305
3 -13.081196 -11.506057  4.999415  137.945408
4  12.702195  -8.147433  5.004363  134.766305
         f0        f1        f2     product
0 -1.146987  0.963328 -0.828965   27.758673
1  0.262778  0.269839 -2.530187   56.069697
2  0.194587  0.289035 -5.586433   62.871910
3  2.236060 -0.553760  0.930038  114.572842
4 -0.515993  1.716266  5.899011  149.600746

Info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 99990 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 

**На этом этапе было выполнено:**
* Созданы переменные для каждого региона
* Исследованны значения в датафреймах
* Данные проверены на наличие пропусков
* Обнаружены и удалены дубликаты по id скважин

#### Создание обучающих выборок

In [9]:
def train_split(data):
    data_train, data_valid = train_test_split(
                        data, test_size=0.25, 
                        random_state=666
                        )
    features_train = data_train.drop(['product'], axis=1)
    target_train = data_train['product']
    features_valid = data_valid.drop(['product'], axis=1)
    target_valid = data_valid['product']
        
    return features_train, target_train, features_valid, target_valid

features_train_one, target_train_one, features_valid_one, target_valid_one = train_split(data1)
features_train_two, target_train_two, features_valid_two, target_valid_two = train_split(data2)
features_train_three, target_train_three, features_valid_three, target_valid_three = train_split(data3)

print(
    'Отношение обучающей выборки к датасету: {:.2f}'.format(features_train_one.shape[0]/data1.shape[0]),
    'Отношение валидационной выборки к датасету: {:.2f}'.format(features_valid_one.shape[0]/data1.shape[0]), sep='\n'
     )

Отношение обучающей выборки к датасету: 0.75
Отношение валидационной выборки к датасету: 0.25


**На этом этапе выполнено:**
* Создана функция для применения train_test_split к датафреймам каждого региона и разделения на признаки
* Выполнена проверка соотношения выборок на примере первой

### Обучение и расчет ошибок модели

In [10]:
def model(features_train, target_train, features_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    return model, predict

model_one, predict_one = model(features_train_one, target_train_one, features_valid_one)
model_two, predict_two = model(features_train_two, target_train_two, features_valid_two)
model_three, predict_three = model(features_train_three, target_train_three, features_valid_three)

In [11]:
def mse(target_valid, predict):
    mse = mean_squared_error(target_valid, predict)
    rmse = mse ** 0.5
    mean_reserve = predict.mean()
    return rmse, mean_reserve

rmse_one, reserve_one = mse(target_valid_one, predict_one)
rmse_two, reserve_two = mse(target_valid_two, predict_two)
rmse_three, reserve_three = mse(target_valid_three, predict_three)

print(
    'Первый регион - {:.0f} средний объем скважин.'.format(reserve_one*10**3), 
    'RMSE:', rmse_one,
    'Второй регион - {:.0f} средний объем скважин.'.format(reserve_two*10**3),
    'RMSE:', rmse_two,
    'Третий регион - {:.0f} средний объем скважин'.format(reserve_three*10**3),
    'RMSE:', rmse_three, sep='\n'
    )

Первый регион - 92187 средний объем скважин.
RMSE:
37.69025980070204
Второй регион - 68946 средний объем скважин.
RMSE:
0.8919858351135961
Третий регион - 94944 средний объем скважин
RMSE:
40.122571305287025


**На этом этапе выполнено:**
* Создана функция для обучения модели линейной регресии на данных трех регионов
* Создана функция для расчета средней квадратичной ошибки предсказаний модели и среднего объема скважины в каждом регионе

**Вывод:**
Второй регион по предсказаниям модели имеет самые низкие показатели среднего объема нефти в скважинах, однако, мы видим, что с данными этого региона на входе модель почти не ошибается. Это значит, что предсказания двух других регионов значительно менее показательны.

### Расчет объема сырья в регионах

In [12]:
invest, barrel, borehole = [10*10**9, 450, 200]
min_production = invest/200/barrel

print(
     'Минимальный объем необходимой выработки одной скважины в регионе составляет: {:.0f} баррелей.'.format(min_production),
     'Сравнение со средним объемом первого региона: {:.0f}'.format(reserve_one*10**3 - min_production),
     'Сравнение со средним объемом второго региона: {:.0f}'.format(reserve_two*10**3 - min_production),
     'Сравнение со средним объемом третьего региона: {:.0f}'.format(reserve_three*10**3 - min_production), sep='\n'
    )                                                                                  

Минимальный объем необходимой выработки одной скважины в регионе составляет: 111111 баррелей.
Сравнение со средним объемом первого региона: -18925
Сравнение со средним объемом второго региона: -42165
Сравнение со средним объемом третьего региона: -16167


In [13]:
def sum_production(prediction):
    production = pd.Series(prediction)
    max_production = production.sort_values(ascending=False)[:200]
    sum_production = max_production.sum()
    return sum_production

sum_one = sum_production(predict_one)
sum_two = sum_production(predict_two)
sum_three = sum_production(predict_three)
    
print(
    'Общий запас в 200 лучших скважинах первого региона: {:.0f} баррелей.'.format(sum_one*10**3),
     'Общий запас в 200 лучших скважинах второго региона: {:.0f} баррелей.'.format(sum_two*10**3),
     'Общий запас в 200 лучших скважинах третьего региона: {:.0f} баррелей.'.format(sum_three*10**3), sep='\n'
)

Общий запас в 200 лучших скважинах первого региона: 30757532 баррелей.
Общий запас в 200 лучших скважинах второго региона: 27741272 баррелей.
Общий запас в 200 лучших скважинах третьего региона: 29791723 баррелей.


**На этом этапе выполнено:**
* Расчитан минимальный объем выработки одной скважины в регионе для безубыточной разработки с двухста скважин
* Создана функция для расчета общего запаса нефти двухста лучших скважин в регионе по предсказанным данным

**Вывод:**
* Если сравнивать необходимый запас в одной скважине и среднее значение, то большая вероятность безубыточности у третьего региона. Однако, после сортировки по убыванию мы видим, что больше всего нефти в скважинах первого региона. При этом разрыв в количестве относительно небольшой, а данные второго региона точнее. 

### Расчет возможной прибыли в регионах по лучшим скважинам

In [14]:
def profit(production):
    profit_region = (production*10**3 * barrel) - invest
    return profit_region

profit_one = profit(sum_one)
profit_two = profit(sum_two)
profit_three = profit(sum_three)

print(
    'Возможная прибыль 200 лучших скважин первого региона: {:.0f}.'.format(profit_one),
     'Возможная прибыль 200 лучших скважин второго региона: {:.0f}.'.format(profit_two),
     'Возможная прибыль 200 лучших скважин третьего региона: {:.0f}.'.format(profit_three), sep='\n'
)

Возможная прибыль 200 лучших скважин первого региона: 3840889206.
Возможная прибыль 200 лучших скважин второго региона: 2483572253.
Возможная прибыль 200 лучших скважин третьего региона: 3406275266.


**На этом этапе выполнено:**
* Создана функция для расчета общей возможной прибыли двухста лучших скважин

**Вывод:**
* По итогам проверки на данном этапе мы видим, что все три региона, согласно предсказаниям для двухста лучших вариантов, могут иметь высокую прибыль. Самая большая прибыль прогнозируется в первом регионе.

### Расчет рисков

In [15]:
prediction_one = pd.Series(predict_one, index=target_valid_one.index)
prediction_two = pd.Series(predict_two, index=target_valid_two.index)
prediction_three = pd.Series(predict_three, index=target_valid_three.index)

In [20]:
def revenue(target, prediction, borehole):
    probs_sorted = prediction.sort_values(ascending=False)
    selected = target[probs_sorted.index][:borehole]
    sum_prod = selected.sum()
    return ((sum_prod * barrel)*10**3) - invest

def bootstrap(target, prediction):
    state = np.random.RandomState(666)
    values = []
    lesion = 0
    
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = prediction[target_subsample.index]
        revenue_get = revenue(target_subsample, probs_subsample, 200)

        if revenue_get < 0:
            lesion += 1
        values.append(revenue_get)
        
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean = values.mean()
    lesion = lesion / 1000
    
    return lower, upper, mean, lesion

lower_one, upper_one, mean_one, lesion_one = bootstrap(target_valid_one, prediction_one)
lower_two, upper_two, mean_two, lesion_two = bootstrap(target_valid_two, prediction_two)
lower_three, upper_three, mean_three, lesion_three = bootstrap(target_valid_three, prediction_three)

**На этом этапе выполнено:**
* Создана функция для расчета рисков и определения верхнего и нижнего квантиля 95% доверительного интервала, а также для просчета среднего значения прибыли двухста случайных скважин в каждом регионе

In [21]:
print(
    'Нижний квантиль 95% доверительного интервала первого региона: {:.0f}.'.format(lower_one),
    'Верхний квантиль 95% доверительного интервала первого региона: {:.0f}.'.format(upper_one),
    '\nНижний квантиль 95% доверительного интервала второго региона: {:.0f}.'.format(lower_two),
    'Верхний квантиль 95% доверительного интервала второго региона: {:.0f}.'.format(upper_two),
    '\nНижний квантиль 95% доверительного интервала третьего региона: {:.0f}.'.format(lower_three), 
    'Верхний квантиль 95% доверительного интервала третьего региона: {:.0f}.'.format(upper_three), sep='\n'
)

Нижний квантиль 95% доверительного интервала первого региона: -36730671.
Верхний квантиль 95% доверительного интервала первого региона: 1032628363.

Нижний квантиль 95% доверительного интервала второго региона: 123032175.
Верхний квантиль 95% доверительного интервала второго региона: 958010510.

Нижний квантиль 95% доверительного интервала третьего региона: -143273177.
Верхний квантиль 95% доверительного интервала третьего региона: 967553576.


In [22]:
print(
    'Среднее значение возможной прибыли первого региона: {:.0f}.'.format(mean_one),
    '\nСреднее значение возможной прибыли второго региона: {:.0f}.'.format(mean_two),
    '\nСреднее значение возможной прибыли третьего региона: {:.0f}.'.format(mean_three), sep='\n'
)

Среднее значение возможной прибыли первого региона: 492013661.

Среднее значение возможной прибыли второго региона: 519322489.

Среднее значение возможной прибыли третьего региона: 383762604.


In [23]:
print(
    'Уровень риска для первого региона: {:.0%}.'.format(lesion_one),
    '\nУровень риска для второго региона: {:.0%}.'.format(lesion_two),
    '\nУровень риска для третьего региона: {:.0%}.'.format(lesion_three), sep='\n'
)

Уровень риска для первого региона: 3%.

Уровень риска для второго региона: 1%.

Уровень риска для третьего региона: 9%.


**Общий вывод:**
По итогам проверки двухста случайных скважин в каждом регионе можно сделать вывод, что **лучшим регионом для разработки является регион номер два**. Мы видим, что именно в этом регионе самый низкий уровень риска убыточности - всего *1%*, а также самое высокое значение возможной средней прибыли - *519 млн.р.*
Кроме этого, второй регион единственный, чей нижний квантиль выше нуля. При этом данные второго региона являются наиболее подходящими для точных предсказаний, согласно расчету средней квадратичной ошибки. 

Самый рискованный **регион третий**, уровень риска здесь целых *9%*, а также третий регион имеет самое низкое среднее значение возможной прибыли. 

**Первый регион**, несмотря на самые большие запасы в двухста лучших скважнах не подходит для разработки, так как имеет уровень риска, превышающий *2.5%*.